In [ ]:
!pip install openai-whisper
!pip install deep-translator
!pip install groq
!pip install gradio
!pip install diffusers transformers gradio accelerate

In [ ]:
# prompt: add the theme  featrue in above  Audio Transcription, Translation, and Sentiment Analysis app and also need selectbox for sentiment and generated image
import whisper
import os
import gradio as gr
from groq import Groq
from deep_translator import GoogleTranslator
import pickle
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt
import torch


# Replace with your actual API key
api_key ="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"
client = Groq(api_key=api_key)

# Load the sentiment analysis model
loaded_model = pickle.load(open('/content/hotel_review_sentiment_model.pkl', 'rb'))

model_id1 = "dreamlike-art/dreamlike-diffusion-1.0"
model_id2 = "stabilityai/stable-diffusion-xl-base-1.0"

pipe = StableDiffusionPipeline.from_pretrained(model_id1, torch_dtype=torch.float16, use_safetensors=True)
pipe = pipe.to("cuda")

prompt = """dreamlikeart, a grungy woman with rainbow hair, travelling between dimensions, dynamic pose, happy, soft eyes and narrow chin,
extreme bokeh, dainty figure, long hair straight down, torn kawaii shirt and baggy jeans
"""
image = pipe(prompt).images[0]

# Function to transcribe, translate, and analyze sentiment
def process_audio(audio_path, sentiment_option, image_option):
    if audio_path is None:
        return "Please upload an audio file.", None, None, None

    # Step 1: Transcribe audio
    try:
        with open(audio_path, "rb") as file:
            transcription = client.audio.transcriptions.create(
                file=(os.path.basename(audio_path), file.read()),
                model="whisper-large-v3",
                language="ta",
                response_format="verbose_json",
            )
        tamil_text = transcription.text
    except Exception as e:
        return f"An error occurred during transcription: {str(e)}", None, None, None

    # Step 2: Translate Tamil to English
    try:
        translator = GoogleTranslator(source='ta', target='en')
        translation = translator.translate(tamil_text)
    except Exception as e:
        return tamil_text, f"An error occurred during translation: {str(e)}", None, None

    # Step 3: Analyze sentiment (if selected)
    sentiment = None
    if sentiment_option == "Analyze Sentiment":
        try:
            sentiment = loaded_model.predict([translation])[0]
        except Exception as e:
            return tamil_text, translation, f"An error occurred during sentiment analysis: {str(e)}", None

    # Step 4: Generate image (if selected)
    image = None
    if image_option == "Generate Image":
        try:
            model_id1 = "dreamlike-art/dreamlike-diffusion-1.0"
            pipe = StableDiffusionPipeline.from_pretrained(model_id1, torch_dtype=torch.float16, use_safetensors=True)
            pipe = pipe.to("cuda")
            image = pipe(translation).images[0]
        except Exception as e:
            return tamil_text, translation, sentiment, f"An error occurred during image generation: {str(e)}"

    return tamil_text, translation, sentiment, image

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Base()) as iface:
    gr.Markdown("# Audio Transcription, Translation, and Sentiment Analysis")
    with gr.Row():
        with gr.Column():
            audio_input = gr.Audio(type="filepath", label="Upload Audio File")
            sentiment_option = gr.Dropdown(["Analyze Sentiment", "Skip Sentiment"], label="Sentiment Analysis", value="Analyze Sentiment")
            image_option = gr.Dropdown(["Generate Image", "Skip Image"], label="Image Generation", value="Generate Image")
            submit_button = gr.Button("Process Audio")
        with gr.Column():
            tamil_text_output = gr.Textbox(label="Tamil Transcription")
            translation_output = gr.Textbox(label="English Translation")
            sentiment_output = gr.Textbox(label="Sentiment")
            image_output = gr.Image(label="Generated Image")

    submit_button.click(
        fn=process_audio,
        inputs=[audio_input, sentiment_option, image_option],
        outputs=[tamil_text_output, translation_output, sentiment_output, image_output]
    )

# Launch the interface
iface.launch()
